In [26]:
import json
import numpy as np
from IPython.display import display, HTML
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [27]:
planner_names = [
    "MTGTaskPlanner",
    "MTGTaskPlannerWithGMM",
#     "DynamicMTGTaskPlannerWithGMM",
#     "DynamicMTGTaskPlannerWithGMM_V2",
    "DynamicMTGTaskPlannerWithGMM_25.0",
    "DynamicMTGTaskPlannerWithGMM_50.0",
#     "DynamicMTGTaskPlannerWithGMM_75.0",
    # "DynamicMTGTaskPlannerWithGMM_100.0",
    "MTGTaskPlannerWithAtlas",
#     "DynamicMTGPlannerWithAtlas",
#     "DynamicMTGPlannerWithAtlas_V2",
    "DynamicMTGPlannerWithAtlas_25.0",
    "DynamicMTGPlannerWithAtlas_50.0",
#     "DynamicMTGPlannerWithAtlas_75.0",
    # "DynamicMTGTaskPlannerWithAtlas_100.0",

]


In [28]:
def read_json(file_path):
    """
    Reads a json file
    """
    result_data = []
    # Open the file and load JSON data
    with open(file_path, 'r') as json_file:
        for line in json_file:
            try:
                result_data.append(json.loads(line))
            except:
                continue
    return result_data
    

In [29]:
filepaths = ["..//check/drive-download-20240114T225505Z-001/open_bottle/check/result.json"]


for file_path in filepaths:
    print(file_path)
    result_data = read_json(file_path)
    planner_stats = {
                        planner_name: {
                            'num' : 0, 
                            'num_success' : [], 
                            'motion_planning_time' : [],
                            'task_planning_time' : [],
                            'updating_time' : [],
                            'solution_length' : [],
                            'num_attempts' : [],
                            'total_planning_time' : [],
                            'set_start_and_goal_time' : [],
                        } 

                     for planner_name in planner_names
                    }

    for d in result_data:
        if d['planner_name'] in planner_stats:
            dic = planner_stats[d['planner_name']]
            dic['num'] += 1
            dic['num_success'].append(d['success'] == "true")
            if d['success'] == "true":
                dic['motion_planning_time'].append(d['motion_planning_time'])
                dic['task_planning_time'].append(d['task_planning_time'])
                dic['total_planning_time'].append(d['total_planning_time'])
                dic['updating_time'].append(d['updating_time'])
                dic['solution_length'].append(d['solution_length'])
                dic['num_attempts'].append(d['num_attempts'])
                dic['set_start_and_goal_time'].append(d['set_start_and_goal_time'])

    output_dict = []
    for planner_name in planner_names:
        stats = planner_stats[planner_name]

        current_planner_eval_dict = {}
        current_planner_eval_dict["Planner Name"] = planner_name
        current_planner_eval_dict["Total Number of Tasks"] = len(stats['num_success'])
        current_planner_eval_dict["Success Rate"] = np.mean(stats['num_success']) * 100.0
        current_planner_eval_dict["Mean Task Planning Time"] = np.mean(np.array(stats['task_planning_time']) / (np.array(stats['num_attempts'])) )
        current_planner_eval_dict["Mean Motion Planning Time"] = np.mean(np.array(stats['motion_planning_time']) / (np.array(stats['num_attempts'])) )
        current_planner_eval_dict["Total Planning Time"] = np.mean(np.array(stats['total_planning_time']) )
        current_planner_eval_dict["Set Start and Goal Time"] = np.mean(np.array(stats['set_start_and_goal_time']) )
        current_planner_eval_dict["Mean Update time"] = np.mean(np.array(stats['updating_time']) / (np.array(stats['num_attempts'])) )
        current_planner_eval_dict["Mean solution length"] = np.mean(np.array(stats['solution_length']) / (np.array(stats['num_attempts'])) )
        current_planner_eval_dict["Mean number of attempts"] = np.mean((np.array(stats['num_attempts'])) )
        output_dict.append(current_planner_eval_dict)
    df = pd.DataFrame(output_dict).set_index('Planner Name')

    display(df)

..//check/drive-download-20240114T225505Z-001/open_bottle/check/result.json


,Mean Motion Planning Time,Mean Task Planning Time,Mean Update time,Mean number of attempts,Mean solution length,Set Start and Goal Time,Success Rate,Total Number of Tasks,Total Planning Time
Planner Name,,,,,,,,,
MTGTaskPlanner,nan,nan,nan,nan,nan,nan,nan,0,nan
MTGTaskPlannerWithGMM,4.2430,0.4306,0.3815,2.0800,12.9501,0.0074,100.0000,50,9.9786
DynamicMTGTaskPlannerWithGMM_25.0,5.7735,0.0034,0.1518,2.8980,15.2368,2.0213,98.0000,50,13.8129
DynamicMTGTaskPlannerWithGMM_50.0,5.7046,0.0005,0.1416,2.7250,14.2696,2.0268,97.5610,41,14.0548
MTGTaskPlannerWithAtlas,3.6603,0.3964,0.3791,3.1400,11.9627,0.0075,100.0000,50,13.7018
DynamicMTGPlannerWithAtlas_25.0,5.4681,0.0050,0.1626,2.2600,13.9521,2.1125,100.0000,50,12.5650
DynamicMTGPlannerWithAtlas_50.0,nan,nan,nan,nan,nan,nan,nan,0,nan
